In [ ]:
## Callin Switzer
## Capture Images from pt grey camera

In [2]:
# setup arduino
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2

import datetime

import PyCapture2 as fc2
import sys

import time

import os
import peakutils 
import msvcrt
import winsound
import shutil
import pandas as pd

import skimage.io as io

In [3]:
print("last update: "  +  str(datetime.datetime.now()))

last update: 2018-01-23 15:50:04.373666


In [4]:
# capture image with point grey camera
bus = fc2.BusManager()
numCams = bus.getNumOfCameras()
print("Number of cameras detected: ", numCams)
if not numCams:
    print("Insufficient number of cameras. Exiting...")
    exit()

Number of cameras detected:  2


In [5]:
def enableEmbeddedTimeStamp(cam, enableTimeStamp):
    embeddedInfo = cam.getEmbeddedImageInfo()
    if embeddedInfo.available.timestamp:
        cam.setEmbeddedImageInfo(timestamp = enableTimeStamp)
        if(enableTimeStamp):
            print("\nTimeStamp is enabled.\n")
        else:
            print("\nTimeStamp is disabled.\n")

In [6]:
# open qtconsole if it's not already running
try:
    console_is_running
except NameError:
    console_is_running = True
    %qtconsole

In [7]:
def printCameraInfo(cam):
    camInfo = cam.getCameraInfo()
    print("\n*** CAMERA INFORMATION ***\n")
    print("Serial number - ", camInfo.serialNumber)
    print("Camera model - ", camInfo.modelName)
    print("Camera vendor - ", camInfo.vendorName)
    print("Sensor - ", camInfo.sensorInfo)
    print("Resolution - ", camInfo.sensorResolution)
    print("Firmware version - ", camInfo.firmwareVersion)
    print("Firmware build time - ", camInfo.firmwareBuildTime)
    fRateProp = cam.getProperty(fc2.PROPERTY_TYPE.FRAME_RATE)
    print("FrameRate - ", fRateProp.absValue)
    print()

In [8]:
c = fc2.Camera()
c.connect(bus.getCameraFromIndex(0))
printCameraInfo(c)

d = fc2.Camera()
d.connect(bus.getCameraFromIndex(1))
printCameraInfo(d)


*** CAMERA INFORMATION ***

Serial number -  16060682
Camera model -  b'Chameleon3 CM3-U3-13Y3C'
Camera vendor -  b'Point Grey Research'
Sensor -  b'OnSemi PYTHON1300 (1/2" Color CMOS)'
Resolution -  b'1280x1024'
Firmware version -  b'1.9.3.0'
Firmware build time -  b'Mon Jul 27 20:45:52 2015'
FrameRate -  92.39653778076172


*** CAMERA INFORMATION ***

Serial number -  16061221
Camera model -  b'Chameleon3 CM3-U3-13Y3C'
Camera vendor -  b'Point Grey Research'
Sensor -  b'OnSemi PYTHON1300 (1/2" Color CMOS)'
Resolution -  b'1280x1024'
Firmware version -  b'1.9.3.0'
Firmware build time -  b'Mon Jul 27 20:45:52 2015'
FrameRate -  92.39653778076172



In [9]:
def img2array(image):
    return(np.array(image.getData(), dtype="uint8").reshape( (image.getRows(), image.getCols()) ))

In [13]:
# start recording video
def saveAviHelper2(cam, cam2, fileFormat, fileName, fileName2, frameRate, maxImgs = 50):
    # clear characters
    while msvcrt.kbhit():
        msvcrt.getch()
        print('clearing characters ...')
    
    numImages = 0

    avi = fc2.AVIRecorder()
    avi2 = fc2.AVIRecorder()

    for i in range(maxImgs):
        
        if msvcrt.kbhit():
            if msvcrt.getch() == b'q':
                # disconnect camera 
    #             c.stop_capture()
    #             c.disconnect()
                print('keyboard q pressed, now quitting loop')
#                 for i in range(5):
#                     winsound.Beep(450,100)
                break   
        try:
            image = cam.retrieveBuffer()
            image2 = cam2.retrieveBuffer()
        except fc2.Fc2error as fc2Err:
            print("Error retrieving buffer : ", fc2Err)
            continue

        print("Grabbed image {}".format(i))

        if (i == 0):
            if fileFormat == "AVI":
                avi.AVIOpen(fileName, frameRate)
                avi2.AVIOpen(fileName2, frameRate)
#             elif fileFormat == "MJPG":
#                 avi.MJPGOpen(fileName, frameRate, 75)
#             elif fileFormat == "H264":
#                 avi.H264Open(fileName, frameRate, image.getCols(), image.getRows(), 1000000)
            else:
                print("Specified format is not available.")
                return

        avi.append(image)
        avi2.append(image2)
        numImages += 1
        print("Appended image {}...".format(i))

    print("Appended {} images to {} file: {}...".format(numImages, fileFormat, fileName))
    avi.close()
    avi2.close()

In [14]:
c = fc2.Camera()
c.connect(bus.getCameraFromIndex(0))
enableEmbeddedTimeStamp(c, True)
c.startCapture()


d = fc2.Camera()
d.connect(bus.getCameraFromIndex(1))
enableEmbeddedTimeStamp(c, True)
d.startCapture()



TimeStamp is enabled.


TimeStamp is enabled.



In [18]:
# display images via live preview

def livePreview2():
    cv2.namedWindow('image',cv2.WINDOW_NORMAL)
    cv2.resizeWindow('image', 1000,800)
    while(True):
        stt = time.time()
        # Capture frame-by-frame
        image = c.retrieveBuffer()
        image2 = d.retrieveBuffer()
        img = np.concatenate((img2array(image), img2array(image2)), axis = 1)

        # Our operations on the frame come here
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Display the resulting frame
        cv2.imshow('image', img)
        
        print(time.time() - stt)

        # break when "q" is pressed on keyboard
        if cv2.waitKey(1) & 0xFF == ord('q'):
            for jj in range(10):
                cv2.destroyAllWindows()
            break



In [20]:
livePreview2()

0.4127929210662842
0.3692629337310791
0.3567538261413574
0.3912796974182129
0.3812718391418457
0.37076425552368164
0.37276387214660645
0.36626100540161133
0.36375927925109863
0.3767683506011963
0.3547523021697998
0.431307315826416
0.4408144950866699
0.4603276252746582
0.4243013858795166
0.3857753276824951
0.3647599220275879
0.344745397567749
0.3957815170288086
0.4318068027496338
0.40428733825683594
0.3912780284881592
0.37326693534851074
0.42830491065979004
0.39177918434143066
0.3722653388977051
0.36826419830322266
0.3522510528564453
0.3722648620605469
0.36926698684692383
0.37326502799987793
0.361757755279541
0.35825490951538086
0.34224390983581543
0.3812713623046875
0.36626100540161133
0.3617575168609619
0.36576080322265625
0.35875463485717773
0.34174346923828125
0.37977075576782227
0.3507497310638428
0.3717648983001709
0.3577544689178467
0.361757755279541
0.3827779293060303
0.3722653388977051
0.36576032638549805
0.3622581958770752
0.34224367141723633
0.3622581958770752
0.3622577190399

In [52]:
import re 
os.path.join(directory, "SaveImageToAviEx2" + re.sub(":", "_", str(datetime.datetime.now())) +".avi")

'C:\\Users\\Combes4\\Desktop\\twoCamSave2\\SaveImageToAviEx22018-01-23 16_19_51.135825.avi'

In [53]:
directory = os.path.join("C:\\Users\\Combes4\Desktop\\twoCamSave2")
fileName = os.path.join(directory, "SaveImageToAviEx1" + re.sub(":", "_", str(datetime.datetime.now())) +".avi")
fileName2 = os.path.join(directory, "SaveImageToAviEx2" + re.sub(":", "_", str(datetime.datetime.now())) +".avi")
saveAviHelper2(c,d, "AVI", fileName.encode("utf-8"), fileName2.encode("utf-8"), 10)

Grabbed image 0
Appended image 0...
Grabbed image 1
Appended image 1...
Grabbed image 2
Appended image 2...
Grabbed image 3
Appended image 3...
Grabbed image 4
Appended image 4...
Grabbed image 5
Appended image 5...
Grabbed image 6
Appended image 6...
Grabbed image 7
Appended image 7...
Grabbed image 8
Appended image 8...
Grabbed image 9
Appended image 9...
Grabbed image 10
Appended image 10...
Grabbed image 11
Appended image 11...
Grabbed image 12
Appended image 12...
Grabbed image 13
Appended image 13...
Grabbed image 14
Appended image 14...
Grabbed image 15
Appended image 15...
Grabbed image 16
Appended image 16...
Grabbed image 17
Appended image 17...
Grabbed image 18
Appended image 18...
Grabbed image 19
Appended image 19...
Grabbed image 20
Appended image 20...
Grabbed image 21
Appended image 21...
Grabbed image 22
Appended image 22...
Grabbed image 23
Appended image 23...
Grabbed image 24
Appended image 24...
Grabbed image 25
Appended image 25...
Grabbed image 26
Appended image 

In [ ]:
# When everything done, release the capture
c.stopCapture()
c.disconnect()

d.stopCapture()
d.disconnect()
cv2.destroyAllWindows()

In [ ]:
os.getcwd()

In [ ]:
fileName

In [31]:
# combine videos to see if they're both the same

vidPath = 'C:\\Users\\Combes4\\Desktop\\twoCamSave2\\SaveImageToAviEx2-0000.avi'
vidPath2 = 'C:\\Users\\Combes4\\Desktop\\twoCamSave2\\SaveImageToAviEx3-0000.avi'

cap = cv2.VideoCapture(vidPath)
cap2 = cv2.VideoCapture(vidPath2)

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

print( length, width, height, fps)

50 1280 1024 10.0


In [32]:

ctr = 0
photoDir = os.path.join("C:\\Users\\Combes4\Desktop\\twoCamSave2\\images\\")

# make directory, if it doesn't exist
if not os.path.exists(photoDir):
    os.makedirs(photoDir)

for ff in range(length):
    ret, frame = cap.read()
    ret2, frame2 = cap2.read()
    im2 = np.concatenate((frame, frame2), axis = 1)
    io.imsave(photoDir + str(ctr).zfill(4) + '.png', im2)
    print(ff)
    ctr += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [30]:
 ret2, frame2 = cap2.read()

In [26]:
io.imsave(photoDir + str(ctr).zfill(4) + '.png', im2)

In [27]:
c1 = [os.path.join(directory, jj) for jj in os.listdir(directory) if jj.startswith("c1")]
c2 = [os.path.join(directory, jj) for jj in os.listdir(directory) if jj.startswith("c2")]

In [28]:
for ii in range(10):
    imgg = np.concatenate((io.imread(c1[ii]), io.imread(c2[ii])), axis = 1)
    plt.figure(figsize=(20,10))
    io.imshow(imgg)

IndexError: list index out of range